<a href="https://colab.research.google.com/github/uob-positron-imaging-centre/PEPT-Algorithms-RoPP/blob/main/PEPTML_RoPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank"  href="https://github.com/uob-positron-imaging-centre/pept"><img src="https://github.com/uob-positron-imaging-centre/misc-hosting/blob/master/logo.png?raw=true" style="height:200px; display: block; margin-left: auto; margin-right: auto;"/></a>

# Interactive PEPT Analysis Examples using *the PEPT-ML algorithm*

> [1] Nicuşan AL, Windows-Yule CR. Positron emission particle tracking using machine learning. Review of Scientific Instruments. 2020 Jan 1;91(1):013329.

---

#### Copyright 2021 the `pept` developers
##### Jupyter Notebook authored by Andrei Leonard Nicusan and Dr. Kit Windows-Yule for the "PEPT: A Comparative Review" paper, commissioned by the Reports on Progress in Physics journal

Licensed under the GNU License, Version 3.0 (the "License").

---


# 1. Introduction

Positron emission particle tracking (PEPT) is a powerful technique allowing the non-invasive, three-dimensional tracking of one or more radioactive 'tracer' particles through particulate, fluid or multiphase systems. It allows particle or fluid motion to be tracked with sub-millimetre accuracy and sub-millisecond temporal resolution and, due to its use of highly-penetrating 511keV gamma rays, can be used to probe the internal dynamics of even large, dense, optically opaque systems <sup>[[2]](https://www.sciencedirect.com/science/article/pii/016890029390864E) [[3]](https://www.sciencedirect.com/science/article/pii/S0263876208003341) [[4]](https://aip.scitation.org/doi/abs/10.1063/1.4983046@rsi.2017.IMGP2017.issue-1)</sup>. In light of its versatility both in terms of the scales and materials of particles which can be tracked <sup>[[5]](https://www.sciencedirect.com/science/article/pii/S1672251507001455)[[6]](https://www.sciencedirect.com/science/article/pii/S0168900206005341)</sup>, and the sizes and geometries of the systems which can be imaged <sup>[[7]](https://www.sciencedirect.com/science/article/pii/S0168900209001880) [[8]](https://www.sciencedirect.com/science/article/pii/S0029549316000273)</sup> , the technique has wide-ranging applicability in diverse scientific, industrial and biomedical applications.

PEPT is performed by radioactively labelling a particle with a positron-emitting radioisotope such as Fluorine-18 ($^{18}\mathrm{F}$) or Gallium-68 ($^{68}\mathrm{Ga}$), and using the back-to-back gamma rays produced by electron-positron annihilation events in and around the tracer to triangulate its spatial position. Each detected gamma ray represents a **line of response (LoR)** .

## 1.1. This Jupyter Notebook

This interactive Jupyter Notebook illustrates the main processing steps employed by the PEPT-ML algorithm<sup>[1]</sup> for radioactive tracer tracking, as described in the Reports on Progress in Physics "PEPT: A Comparative Review" paper.

An [example dataset](https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv) is used from an experiment run at the University of Birmingham Positron Imaging Centre using the ADAC Forté by Matthew Herald. It consists of a single 1 mm diameter MCC particle activated with Fluorine-18 radioactive tracer material inside a bubbling fluidised bed. The fluidised bed was filled with 90% sand and 10% MCC; air was fed into the bottom of the bed at a rate of 37 litres per minute at 3.5 bar. This dataset was chosen for its high quality captured lines of response, with the tracer still depicting the random particle motion that is inherent to bubbling fluidised beds - and typical in Lagrangian particle tracking.

The [`pept`](https://github.com/uob-positron-imaging-centre/pept) Python library is used for initialising and visualising PEPT data. While not required *per se* for illustrating PEPT algorithms' processing steps, it significantly reduces the amount of repetitive code and visual noise, allowing the reader to focus on the main conceptual procedures.

## 1.2. Running Code Cells
Select any code cell and click on the (▶) sign in the top-left of the cell's frame to run its code (note that code cells have to be run in order when running for the first time).

In [22]:
# First install the `pept` library using pip, Python's package manager
!pip install pept

# 2. The PEPT-ML Algorithm

## 2.1. Read in Line of Response Data
In this initial step, we read in the "raw" LoR data from which our tracer positions are calculated. In this simple example, we load only a single sample (i.e. one particle location at one point in time). The LoRs corresponding to this sample are output as an image.

As discussed in the main text, varying the numbers of LoRs used to calculate a PEPT location can affect the quality of the final location measured. Try altering the value "nrows" below to see how this inluences your results.

In [23]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 200,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors = np.insert(lors_raw, 6, head_separation, axis = 1)

# Print the line of response (LoR) data
lors

array([[0.000e+00, 1.900e+02, 1.687e+02, ..., 3.463e+02, 1.428e+02,
        6.000e+02],
       [1.000e-01, 2.437e+02, 1.676e+02, ..., 3.145e+02, 3.139e+02,
        6.000e+02],
       [1.000e-01, 1.941e+02, 4.100e+02, ..., 4.171e+02, 2.401e+02,
        6.000e+02],
       ...,
       [7.800e+00, 1.351e+02, 3.009e+02, ..., 4.366e+02, 1.870e+02,
        6.000e+02],
       [7.800e+00, 3.168e+02, 5.174e+02, ..., 4.195e+02, 2.100e+02,
        6.000e+02],
       [8.000e+00, 2.767e+02, 1.658e+02, ..., 2.938e+02, 2.024e+02,
        6.000e+02]])

In [24]:
from pept.plots import PlotlyGrapher

grapher = PlotlyGrapher()
grapher.add_lines(lors)
grapher.show()

## 2.2. Find Cutpoints
In this step, we find the point of closest approach for each pair of LoRs and take the midpoint between the two lines at this point as a 'cutpoint'.

Of course, for the purposes of PEPT, we only want to do this for LoRs that are being emitted from our tracer, and ignore false LoRs as far as possible. We do this by defining a `max_distance`, beyond which we can assume two LoRs do not pass through our particle, and simply not calculating a cutpoint for any two LoRs separated by more that this distance.

Try adjusting the value of `max_distance` and see what effect it has on the distribution of cutpoints produced.

As with the `nlors_remove` parameter in the Birmingham method example, being more restrictive with your choice of `max_distance` will remove more erroneous points, but yield less data, while being less restrictive will give you more data, but a higher probability of including erroneous values.

In [25]:
# For all pairs of lines closer than `max_distance`, find their minimum distance points (cutpoints)
from pept.tracking import peptml

max_distance = 0.15
cutpoints = peptml.find_cutpoints(lors, max_distance)

In [26]:
grapher = PlotlyGrapher(cols = 2)

grapher.add_lines(lors)
grapher.add_points(cutpoints, col = 2)

grapher.show()

## 2.3. Cluster Cutpoints with HDBSCAN
Having converted our LoRs into cutpoints, the next step is to create one or more clusters from these cutpoints using the HDBSCAN algorithm, details of which you can find in the appendices of the main article.

In order to optimise the clustering procedure, you can adjust the  `min_cluster_size` parameter, which sets the minimum number of cutpoints that have to form a high-density region in space in order for the algorithm to view them as a true cluster. A larger value of `min_cluster_size` will give harsher clustering, with more points being classified as noise.

Try some different values and see the influence it has on your results.

In [27]:
# Use the HDBSCAN algorithm to find clusters in the cutpoints
from hdbscan import HDBSCAN

min_cluster_size = 20
min_samples = 20

hdbscan = HDBSCAN(min_cluster_size, min_samples, allow_single_cluster = True)
labels = hdbscan.fit_predict(cutpoints.points[:, 1:4])

labels

array([-1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1,  0, -1, -1, -1,  0, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1,  0, -1, -1, -1,
       -1, -1,  0, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,
       -1,  0, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,  0,
        0, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1,  0, -1,  0, -1,
       -1, -1,  0, -1,  0])

In [28]:
# Plot cutpoints, coloured by their label index (numbered from 0, with -1 representing noise)
grapher = PlotlyGrapher()
grapher.add_points(cutpoints, color = labels)
grapher.show()

## 2.4. Compute Cluster Centres
The final step in this basic illustration of the PEPT-ML algorithm is to find the centres of these clusters, which should - if we have chosen our parameters correctly - correspond to the centre(s) of our tracer(s).

In the real PEPT-ML algorithm, we would now also have the option to produce multiple such centres, and then repeat the clustering process with these. If you are interested in looking in more detail at this process, you can find a guide to the full, open-source PEP-ML code [here](https://colab.research.google.com/drive/1G8XHP9zWMMDVu23PXzANLCOKNP_RjBEO#scrollTo=EuxmvtwouHwB).

In [29]:
# Extract cutpoints from the first cluster
cluster = cutpoints.points[labels == 0]
centre = cluster.mean(axis = 0)

In [30]:
grapher = PlotlyGrapher()

grapher.add_points(cutpoints, color = labels)
grapher.add_points([centre], size = 10)

grapher.show()

# 3. Complete PEPT-ML Code

In [31]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 80_000,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors = np.insert(lors_raw, 6, head_separation, axis = 1)

# Print the line of response (LoR) data
lors

array([[0.0000e+00, 1.9000e+02, 1.6870e+02, ..., 3.4630e+02, 1.4280e+02,
        6.0000e+02],
       [1.0000e-01, 2.4370e+02, 1.6760e+02, ..., 3.1450e+02, 3.1390e+02,
        6.0000e+02],
       [1.0000e-01, 1.9410e+02, 4.1000e+02, ..., 4.1710e+02, 2.4010e+02,
        6.0000e+02],
       ...,
       [2.6443e+03, 3.9650e+02, 2.2480e+02, ..., 2.3420e+02, 3.4280e+02,
        6.0000e+02],
       [2.6444e+03, 1.6340e+02, 1.3690e+02, ..., 2.0650e+02, 1.4750e+02,
        6.0000e+02],
       [2.6444e+03, 1.5580e+02, 5.4220e+02, ..., 3.2330e+02, 5.5280e+02,
        6.0000e+02]])

In [32]:
# Use PEPT-ML to track moving tracer
sample_start = 0
sample_size = 200
overlap = 100

max_distance = 0.1
min_cluster_size = 20
min_samples = 20

hdbscan = HDBSCAN(min_cluster_size, min_samples, allow_single_cluster = True)
positions = []

while sample_start + sample_size < len(lors):
    sample = lors[sample_start:sample_start + sample_size]

    cutpoints = peptml.find_cutpoints(sample, max_distance)
    labels = hdbscan.fit_predict(cutpoints.points[:, 1:4])

    for label in range(labels.max() + 1):
        cluster = cutpoints.points[labels == label]
        centre = cluster.mean(axis = 0)
        positions.append(centre)

    sample_start += sample_size - overlap

positions = np.array(positions)
positions

array([[   4.7725    ,  279.56800914,  244.92244165,  319.14862861],
       [   8.27      ,  280.29579   ,  244.04033748,  318.08396057],
       [  12.195     ,  280.81941024,  243.28373819,  316.74611124],
       ...,
       [2631.4775    ,  293.61201411,  292.95853517,  305.8847119 ],
       [2635.4375    ,  293.81850254,  292.92567713,  307.94976094],
       [2638.425     ,  294.10798464,  292.70281621,  307.53975116]])

In [33]:
# Re-cluster the `positions` already found to "tighten" the tracer trajectory. Use a much smaller
# sample size and maximum overlap to minimise temporal resolution loss
sample_start = 0
sample_size = 30
overlap = 29

min_cluster_size = 15
min_samples = 15

hdbscan = HDBSCAN(min_cluster_size, min_samples, allow_single_cluster = True)
positions_2pass = []

while sample_start + sample_size < len(positions):
    sample = positions[sample_start:sample_start + sample_size]     # Directly cluster `positions`
    labels = hdbscan.fit_predict(sample[:, 1:4])

    for label in range(labels.max() + 1):
        cluster = sample[labels == label]
        centre = cluster.mean(axis = 0)
        positions_2pass.append(centre)

    sample_start += sample_size - overlap

positions_2pass = np.array(positions_2pass)
positions_2pass

array([[  45.90365873,  280.56259458,  245.02582085,  316.75028113],
       [  44.76065873,  280.41593563,  244.95390113,  316.55735953],
       [  45.4783254 ,  280.36793955,  244.93381463,  316.43895925],
       ...,
       [2580.52366667,  291.08171993,  291.75963447,  302.71670969],
       [2580.52366667,  291.08171993,  291.75963447,  302.71670969],
       [2581.99833333,  291.2812943 ,  291.7468531 ,  302.77575737]])

In [34]:
grapher = PlotlyGrapher(cols = 3)

grapher.add_lines(lors[:400])
grapher.add_points(positions, col = 2)
grapher.add_points(positions_2pass, col = 3)

grapher.show()